In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
import pyterrier as pt
if not pt.java.started():
    pt.java.init()

import os, sys
os.chdir('/mnt/primary/exposure-fairness')
import config, fair_utils, convert

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
msmarco-passage/dev documents: 100%|██████████| 8841823/8841823 [00:26<00:00, 332692.88it/s]


In [175]:
import importlib
# importlib.reload(config)
# importlib.reload(fair_utils)
importlib.reload(convert)

<module 'convert' from '/mnt/primary/exposure-fairness/convert.py'>

In [97]:
modelname = "splade"
granu = 10000
km = config.kmeans_vec[0]
res_path = f'{config.data_dir}/{modelname}_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.res'
print(res_path)
print(f'processing {res_path}')
doc_df = fair_utils.convert_res2df(res_path)

query_res_path = f'{config.data_dir}/grouped_queries/clustered_dev_queries_by_{granu}_{km}.res'
q_df = fair_utils.convert_query_res2df(query_res_path)
merged_df = doc_df.merge(q_df[['qid','cluster']], on='qid', how='left')

run_name = os.path.splitext(res_path)[0].split('/')[-1] + '_with_groupid'
merged_file_path = f'{config.data_dir}/{run_name}.res'
fair_utils.convert_df2trec(merged_df,merged_file_path)

processing /mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res
converting /mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res into a dataframe
done
converting /mnt/datasets/cxj/exposure-fairness/grouped_queries/clustered_dev_queries_by_10000_scikit_dense.res into a dataframe
done
saving into /mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100_with_groupid.res
saved


'/mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100_with_groupid.res'

In [98]:
doc_df.head()

,qid,Q0,docid,rank,score,run,docno,query
0,1048578,Q0,7187234,0,1345.927870,splade_msmarco-passage_dev_100,7187234,cost of endless pools swim spa
1,1048578,Q0,2078221,1,1324.328383,splade_msmarco-passage_dev_100,2078221,cost of endless pools swim spa
2,1048578,Q0,5365326,2,1322.672730,splade_msmarco-passage_dev_100,5365326,cost of endless pools swim spa
3,1048578,Q0,4567130,3,1308.076251,splade_msmarco-passage_dev_100,4567130,cost of endless pools swim spa
4,1048578,Q0,5365328,4,1250.860573,splade_msmarco-passage_dev_100,5365328,cost of endless pools swim spa


In [99]:
q_df.head()

,qid,query,cluster
0,1048578,cost of endless pools swim spa,5123
1,1048579,what is pcnt,2941
2,1048580,what is pcb waste,7728
3,1048581,what is pbis,1232
4,1048582,what is paysky,4445


In [110]:
# q_df.shape
# doc_df.shape
# q_df[['qid', 'cluster']].head()
# q_df[q_df['query'] == '']
topics = config.topics
topics[topics['qid'] == '388892']



,qid,query
53300,388892,


In [104]:
merged_df = doc_df.merge(q_df[['qid', 'cluster']], on='qid', how='left')

In [107]:
merged_df[merged_df['query']== ''].head()

,qid,Q0,docid,rank,score,run,docno,query,cluster
5330000,388892,Q0,8332088,0,1019.864300,splade_msmarco-passage_dev_100,8332088,,NaN
5330001,388892,Q0,5324244,1,949.213740,splade_msmarco-passage_dev_100,5324244,,NaN
5330002,388892,Q0,8332089,2,926.069976,splade_msmarco-passage_dev_100,8332089,,NaN
5330003,388892,Q0,7864268,3,919.262918,splade_msmarco-passage_dev_100,7864268,,NaN
5330004,388892,Q0,4132200,4,889.437471,splade_msmarco-passage_dev_100,4132200,,NaN


In [136]:
topics = config.topics
topics[topics['query'] == '']

,qid,query
53300,388892,


In [158]:
:query_path = f'{config.data_dir}/grouped_queries/clustered_dev_queries_by_{granu}_{km}.csv'
print(query_path)
query_df = pd.read_csv(query_path, index_col=0).reset_index()

doc_path = '/mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res'
print(doc_path)
doc_df = convert.convert_res2docdf(doc_path)

merged_df2 = doc_df.merge(query_df[['qid','cluster']], on='qid', how='left')


/mnt/datasets/cxj/exposure-fairness/grouped_queries/clustered_dev_queries_by_10000_scikit_dense.csv
/mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res
converting /mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res into a dataframe
done


In [161]:
csv = f'{config.data_dir}/{modelname}_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.csv'
df = pd.read_csv(csv, index_col=0).reset_index()

In [162]:
df = df.sort_values(by=['qid'])

,qid,docid,docno,score,rank
0,1048578,7187234,7187234,1345.927870,0
1,1048578,2078221,2078221,1324.328383,1
2,1048578,5365326,5365326,1322.672730,2
3,1048578,4567130,4567130,1308.076251,3
4,1048578,5365328,5365328,1250.860573,4


In [159]:
# query_df.head()
query_df[query_df['qid'] == 388892]

,qid,query,cluster
53300,388892,NaN,9989


In [190]:
merged_df2[(merged_df2['qid'] == 388892) ]

,qid,Q0,docid,rank,score,run,cluster
5330000,388892,Q0,8332088,0,1019.864300,splade_msmarco-passage_dev_100,9989
5330001,388892,Q0,5324244,1,949.213740,splade_msmarco-passage_dev_100,9989
5330002,388892,Q0,8332089,2,926.069976,splade_msmarco-passage_dev_100,9989
5330003,388892,Q0,7864268,3,919.262918,splade_msmarco-passage_dev_100,9989
5330004,388892,Q0,4132200,4,889.437471,splade_msmarco-passage_dev_100,9989
5330005,388892,Q0,315209,5,877.450503,splade_msmarco-passage_dev_100,9989
5330006,388892,Q0,5434018,6,868.961125,splade_msmarco-passage_dev_100,9989
5330007,388892,Q0,8332090,7,866.271324,splade_msmarco-passage_dev_100,9989
5330008,388892,Q0,5526580,8,863.445983,splade_msmarco-passage_dev_100,9989
5330009,388892,Q0,1408213,9,861.304322,splade_msmarco-passage_dev_100,9989


In [151]:
merged_df2.head()

,qid,docid,docno,score,rank,cluster
0,1048578,7187234,7187234,1345.927870,0,5123
1,1048578,2078221,2078221,1324.328383,1,5123
2,1048578,5365326,5365326,1322.672730,2,5123
3,1048578,4567130,4567130,1308.076251,3,5123
4,1048578,5365328,5365328,1250.860573,4,5123


TypeError: convert_res2docdf() got an unexpected keyword argument 'columns'

In [167]:
df.head()

,qid,Q0,docid,rank,score,run,cluster
0,199177,Q0,6694981,97,10.922701,bm25_msmarco-passage_dev_100,0
1,743456,Q0,3609042,4,25.286708,bm25_msmarco-passage_dev_100,0
2,743456,Q0,3609038,3,26.386765,bm25_msmarco-passage_dev_100,0
3,743456,Q0,2376243,2,26.572051,bm25_msmarco-passage_dev_100,0
4,743456,Q0,7589344,1,26.984184,bm25_msmarco-passage_dev_100,0


In [168]:
res_path = f'{config.data_dir}/bm25_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.res'
print(f'processing {res_path}')
doc_df = convert.convert_res2docdf(res_path)

query_path = f'{config.data_dir}/grouped_queries/clustered_dev_queries_by_500_scikit_dense.csv'
query_df = pd.read_csv(query_path, index_col=0).reset_index()
merged_df = doc_df.merge(query_df[['qid', 'cluster']], on='qid', how='left')
merged_df = merged_df.sort_values(by=['cluster'])

processing /mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100.res
converting /mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100.res into a dataframe
done


In [172]:
query_df.columns

Index(['qid', 'query', 'cluster'], dtype='object')

In [171]:
doc_df.columns

Index(['qid', 'Q0', 'docid', 'rank', 'score', 'run'], dtype='object')

In [170]:
merged_df.columns

Index(['qid', 'Q0', 'docid', 'rank', 'score', 'run', 'cluster'], dtype='object')

In [22]:
# csv = '/mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()
csv = f'{config.data_dir}/bm25_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.csv'
df = pd.read_csv(csv, index_col=0).reset_index()

In [23]:
# df[(df['qid'] == 388892)]
df.head()

,qid,docid,docno,score,rank,query
0,1048578,7187236,7187236,67.528593,0,cost of endless pools swim spa
1,1048578,7471198,7471198,62.121928,1,cost of endless pools swim spa
2,1048578,5365326,5365326,51.958492,2,cost of endless pools swim spa
3,1048578,7187241,7187241,51.947947,3,cost of endless pools swim spa
4,1048578,6802210,6802210,47.925985,4,cost of endless pools swim spa


In [29]:
res = f'{config.data_dir}/splade_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.res'
print(res)
columns=["qid", "Q0", "docid", "rank", "score", "run"]
df = convert.convert_res2docdf(res, columns=columns)

/mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res
converting /mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.res into a dataframe
done


In [31]:
df.head()

,qid,Q0,docid,rank,score,run
0,1048578,Q0,7187234,0,1345.927870,splade_msmarco-passage_dev_100
1,1048578,Q0,2078221,1,1324.328383,splade_msmarco-passage_dev_100
2,1048578,Q0,5365326,2,1322.672730,splade_msmarco-passage_dev_100
3,1048578,Q0,4567130,3,1308.076251,splade_msmarco-passage_dev_100
4,1048578,Q0,5365328,4,1250.860573,splade_msmarco-passage_dev_100


In [32]:
res = '/mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100_500_scikit_dense_with_groupid.res'
columns=["qid", "Q0", "docid", "rank", "score", "run", "cluster"]
df = convert.convert_res2docdf(res, columns=columns)

converting /mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100_500_scikit_dense_with_groupid.res into a dataframe
done


In [33]:
df.head()

,qid,Q0,docid,rank,score,run,cluster
0,199177,Q0,6694981,97,10.922701,bm25_msmarco-passage_dev_100,0
1,743456,Q0,3609042,4,25.286708,bm25_msmarco-passage_dev_100,0
2,743456,Q0,3609038,3,26.386765,bm25_msmarco-passage_dev_100,0
3,743456,Q0,2376243,2,26.572051,bm25_msmarco-passage_dev_100,0
4,743456,Q0,7589344,1,26.984184,bm25_msmarco-passage_dev_100,0


In [34]:
# df[df['qid'] == 388892]
df[(df['qid'] == 388892) ]

,qid,Q0,docid,rank,score,run,cluster


In [78]:
res = '/mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100_500_scikit_dense_with_groupid.res'
columns=["qid", "Q0", "docid", "rank", "score", "run", "cluster"]
df2 = convert.convert_res2docdf(res, columns=columns)
# columns=["qid", "Q0", "docid", "rank", "score", "run"]
# df2 = convert.convert_res2docdf(res, columns=columns)
df2[df2['qid'] == 388892]

converting /mnt/datasets/cxj/exposure-fairness/bm25_msmarco-passage_dev_100_500_scikit_dense_with_groupid.res into a dataframe
done


,qid,Q0,docid,rank,score,run,cluster


In [80]:
df2.tail()

,qid,Q0,docid,rank,score,run,cluster
10108430,1060109,Q0,8839182,22,17.568175,bm25_msmarco-passage_dev_100,499
10108431,1060109,Q0,1442087,23,17.426964,bm25_msmarco-passage_dev_100,499
10108432,1060109,Q0,3795147,24,17.395200,bm25_msmarco-passage_dev_100,499
10108433,1060109,Q0,7168913,25,17.271212,bm25_msmarco-passage_dev_100,499
10108434,1060109,Q0,4031002,26,17.087777,bm25_msmarco-passage_dev_100,499


In [69]:
df.head()

,qid,Q0,docid,rank,score,run,cluster
0,766473,Q0,4725852,83,537.166061,splade_msmarco-passage_dev_100,0
1,766473,Q0,866130,82,540.176600,splade_msmarco-passage_dev_100,0
2,766473,Q0,12460,81,541.075366,splade_msmarco-passage_dev_100,0
3,766473,Q0,578936,80,541.648549,splade_msmarco-passage_dev_100,0
4,766473,Q0,3666452,99,520.578600,splade_msmarco-passage_dev_100,0


In [70]:
df2.head()

,qid,Q0,docid,rank,score,run
766473,Q0,4725852,83,537.166061,splade_msmarco-passage_dev_100,0
766473,Q0,866130,82,540.176600,splade_msmarco-passage_dev_100,0
766473,Q0,12460,81,541.075366,splade_msmarco-passage_dev_100,0
766473,Q0,578936,80,541.648549,splade_msmarco-passage_dev_100,0
766473,Q0,3666452,99,520.578600,splade_msmarco-passage_dev_100,0


In [64]:
df2[df2['qid'] == 388892]

,qid,Q0,docid,rank,score,run,cluster
7542016,388892,Q0,8332088,0,1019.864300,splade_msmarco-passage_dev_100,364
7542017,388892,Q0,5324244,1,949.213740,splade_msmarco-passage_dev_100,364
7542018,388892,Q0,8332089,2,926.069976,splade_msmarco-passage_dev_100,364
7542019,388892,Q0,7864268,3,919.262918,splade_msmarco-passage_dev_100,364
7542020,388892,Q0,4132200,4,889.437471,splade_msmarco-passage_dev_100,364
7542021,388892,Q0,315209,5,877.450503,splade_msmarco-passage_dev_100,364
7542022,388892,Q0,5434018,6,868.961125,splade_msmarco-passage_dev_100,364
7542023,388892,Q0,8332090,7,866.271324,splade_msmarco-passage_dev_100,364
7542024,388892,Q0,5526580,8,863.445983,splade_msmarco-passage_dev_100,364
7542025,388892,Q0,1408213,9,861.304322,splade_msmarco-passage_dev_100,364


In [48]:
csv = f'{config.data_dir}/splade_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}.csv'
print(csv)
df = pd.read_csv(csv, index_col=0).reset_index()

/mnt/datasets/cxj/exposure-fairness/splade_msmarco-passage_dev_100.csv


In [39]:
df.dtypes

qid        int64
Q0        object
docid      int64
rank       int64
score    float64
run       object
dtype: object

In [49]:
q = '/mnt/datasets/cxj/exposure-fairness//grouped_queries/clustered_dev_queries_by_500_scikit_dense.csv'
q_df = pd.read_csv(q, index_col=0).reset_index()

In [52]:
m_df = df.merge(q_df[['qid','cluster']], on='qid', how='left')

In [ ]:
m_df[(m_df['qid'] == 388892)]

In [ ]:
df[(df['qid'] == 388892) ]